In [3]:
import os

from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from typing_extensions import Annotated, Literal, TypedDict

from langgraph.graph import add_messages


_ = load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "")

class Router(BaseModel):
    """Analyze the unread email and route it according to its content.
    """
    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, 'notify' for important information that doesn't need a response, 'respond' for emails that need a reply",
    )

class State(TypedDict):
    email_input: str
    messages: Annotated[List, add_messages]
    
profile = {
    "name": "Yoona",
    "full_name": "Lim Yoona",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [4]:
from typing import List

from langchain_core.tools import tool
from langchain_openai import OpenAIEmbeddings

from langgraph.store.memory import InMemoryStore

from langmem import create_manage_memory_tool, create_search_memory_tool


@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'."

@tool
def schedule_meeting(
    attendees: List[str], 
    subject: str, 
    duration_minutes: int, 
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees."

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM."

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

store = InMemoryStore(
    index={"embed": embeddings}
)

manage_memory_tool = create_manage_memory_tool(
    namespace=(
        "email_assistant", 
        "{langgraph_user_id}",
        "collection"
    )
)
search_memory_tool = create_search_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)

In [5]:
from prompts import agent_system_prompt_memory_2, prompt_instructions


def create_prompt(state, config, store):
    langgraph_user_id = config['configurable']['langgraph_user_id']
    namespace = (langgraph_user_id, )
    result = store.get(namespace, "agent_instructions")
    if result is None:
        store.put(
            namespace, 
            "agent_instructions", 
            {"prompt": prompt_instructions["agent_instructions"]}
        )
        prompt = prompt_instructions["agent_instructions"]
    else:
        prompt = result.value['prompt']
        
    return [
        {
            "role": "system", 
            "content": agent_system_prompt_memory_2.format(
                full_name=profile["full_name"],
                name=profile["name"],
				instructions=prompt, 
            )
        }
    ] + state['messages']

In [6]:
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
	model="openai/gpt-4.1",
	base_url="https://openrouter.ai/api/v1",
	api_key=OPENROUTER_API_KEY
)

tools= [
    write_email, 
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool
]

response_agent = create_react_agent(
    llm,
    tools=tools,
    prompt=create_prompt,
    store=store	    # use this to ensure the store is passed to the agent 
)

In [7]:
# template for formating an example to put in prompt
template = """
	Email Subject: {subject}
	Email From: {from_email}
	Email To: {to_email}
	Email Content: 
	'''
	{content}
	'''
	> Triage Result: {result}
"""

# format list of few shots
def format_few_shot_examples(examples):
    strs = ["Here are some previous examples:"]
    for eg in examples:
        strs.append(
            template.format(
                subject=eg.value["email"]["subject"],
                to_email=eg.value["email"]["to"],
                from_email=eg.value["email"]["author"],
                content=eg.value["email"]["email_thread"][:400],
                result=eg.value["label"],
            )
        )
    return "\n\n------------\n\n".join(strs)

In [8]:
from typing import Literal

from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command

from prompts import triage_system_prompt, triage_user_prompt, prompt_instructions


llm = ChatOpenAI(
	model="openai/gpt-4.1",
	base_url="https://openrouter.ai/api/v1",
	api_key=OPENROUTER_API_KEY
)
llm_router = llm.with_structured_output(Router)

def triage_router(state: State, config, store) -> Command[Literal["response_agent", "__end__"]]:
	author = state['email_input']['author']
	to = state['email_input']['to']
	subject = state['email_input']['subject']
	email_thread = state['email_input']['email_thread']

	namespace = ("email_assistant", config['configurable']['langgraph_user_id'], "examples")
	examples = store.search(
		namespace, 
		query=str({"email": state['email_input']})
	) 
	examples=format_few_shot_examples(examples)

	langgraph_user_id = config['configurable']['langgraph_user_id']
	namespace = (langgraph_user_id, )

	result = store.get(namespace, "triage_ignore")
	if result is None:
		store.put(
			namespace, 
			"triage_ignore", 
			{"prompt": prompt_instructions["triage_rules"]["ignore"]}
		)
		ignore_prompt = prompt_instructions["triage_rules"]["ignore"]
	else:
		ignore_prompt = result.value['prompt']

	result = store.get(namespace, "triage_notify")
	if result is None:
		store.put(
			namespace, 
			"triage_notify", 
			{"prompt": prompt_instructions["triage_rules"]["notify"]}
		)
		notify_prompt = prompt_instructions["triage_rules"]["notify"]
	else:
		notify_prompt = result.value['prompt']

	result = store.get(namespace, "triage_respond")
	if result is None:
		store.put(
			namespace, 
			"triage_respond", 
			{"prompt": prompt_instructions["triage_rules"]["respond"]}
		)
		respond_prompt = prompt_instructions["triage_rules"]["respond"]
	else:
		respond_prompt = result.value['prompt']

	system_prompt = triage_system_prompt.format(
		full_name=profile["full_name"],
		name=profile["name"],
		user_profile_background=profile["user_profile_background"],
		triage_no=ignore_prompt,
		triage_notify=notify_prompt,
		triage_email=respond_prompt,
		examples=examples
	)
	user_prompt = triage_user_prompt.format(
		author=author, 
		to=to, 
		subject=subject, 
		email_thread=email_thread
	)
	result = llm_router.invoke(
		[
			{"role": "system", "content": system_prompt},
			{"role": "user", "content": user_prompt},
		]
	)
	if result.classification == "respond":
		print("📧 Classification: RESPOND - This email requires a response")
		goto = "response_agent"
		update = {
			"messages": [
				{
					"role": "user",
					"content": f"Respond to the email {state['email_input']}",
				}
			]
		}
	elif result.classification == "ignore":
		print("🚫 Classification: IGNORE - This email can be safely ignored")
		update = None
		goto = END
	elif result.classification == "notify":
		# if real life, this would do something else
		print("🔔 Classification: NOTIFY - This email contains important information")
		update = None
		goto = END
	else:
		raise ValueError(f"Invalid classification: {result.classification}")
	return Command(goto=goto, update=update)

email_agent = StateGraph(State)
email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")
email_agent = email_agent.compile(store=store)

In [9]:
# current behaviour

email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "Lim Yoona <yoona@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """
    	Hi Yoon,

		Urgent issue - your service is down. Is there a reason why.
    """,
}

response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "lance"}}
)

📧 Classification: RESPOND - This email requires a response


In [10]:
# look at current values of long term memory

store.get(("lance",), "agent_instructions").value['prompt']

"Use these tools when appropriate to help manage Yoona's tasks efficiently."

In [11]:
store.get(("lance",), "triage_respond").value['prompt']

'Direct questions from team members, meeting requests, critical bug reports'

In [12]:
store.get(("lance",), "triage_ignore").value['prompt']

'Marketing newsletters, spam emails, mass company announcements'

In [13]:
store.get(("lance",), "triage_notify").value['prompt']

'Team member out sick, build system notifications, project status updates'

In [14]:
# now use LLM to update instructions

from langmem import create_multi_prompt_optimizer


conversations = [
    (
        response['messages'],
        "Always sign your emails 'Lim Yoona'"
    )
]

prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
        
    },
    {
        "name": "triage-ignore", 
        "prompt": store.get(("lance",), "triage_ignore").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"

    },
    {
        "name": "triage-notify", 
        "prompt": store.get(("lance",), "triage_notify").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of"

    },
    {
        "name": "triage-respond", 
        "prompt": store.get(("lance",), "triage_respond").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to"

    },
]

optimizer = create_multi_prompt_optimizer(
    llm,
    kind="prompt_memory",
)

updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)
updated

[{'name': 'main_agent',
  'prompt': "Use these tools when appropriate to help manage Yoona's tasks efficiently.\nAlways sign emails as 'Lim Yoona'.",
  'update_instructions': 'keep the instructions short and to the point',
  'when_to_update': 'Update this prompt whenever there is feedback on how the agent should write emails or schedule events'},
 {'name': 'triage-ignore',
  'prompt': 'Marketing newsletters, spam emails, mass company announcements',
  'update_instructions': 'keep the instructions short and to the point',
  'when_to_update': 'Update this prompt whenever there is feedback on which emails should be ignored'},
 {'name': 'triage-notify',
  'prompt': 'Team member out sick, build system notifications, project status updates',
  'update_instructions': 'keep the instructions short and to the point',
  'when_to_update': 'Update this prompt whenever there is feedback on which emails the user should be notified of'},
 {'name': 'triage-respond',
  'prompt': 'Direct questions from t

In [15]:
# update prompts in store

for i, updated_prompt in enumerate(updated):
    old_prompt = prompts[i]
    if updated_prompt['prompt'] != old_prompt['prompt']:
        name = old_prompt['name']
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("lance",),
                "agent_instructions",
                {"prompt":updated_prompt['prompt']}
            )
        else:
            print(f"Encountered {name}, implement the remaining stores!")

updated main_agent


In [16]:
store.get(("lance",), "agent_instructions").value['prompt']

"Use these tools when appropriate to help manage Yoona's tasks efficiently.\nAlways sign emails as 'Lim Yoona'."

In [17]:
# current behaviour

email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "Lim Yoona <yoona@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """
    	Hi Yoon,

		Urgent issue - your service is down. Is there a reason why.
    """,
}

response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "lance"}}
)
for m in response["messages"]:
    m.pretty_print()

📧 Classification: RESPOND - This email requires a response
================================ Human Message =================================

Respond to the email {'author': 'Alice Jones <alice.jones@bar.com>', 'to': 'Lim Yoona <yoona@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': '\n    \tHi Yoon,\n\n\t\tUrgent issue - your service is down. Is there a reason why.\n    '}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_dzqjlcwrSU7r5FXojcvxqmIK)
 Call ID: call_dzqjlcwrSU7r5FXojcvxqmIK
  Args:
    to: alice.jones@bar.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thank you for letting me know. I am looking into the issue right now and will update you as soon as I have more information. Please let me know if you have any additional details about the downtime that may help with troubleshooting.

Best regards,
Lim Yoona
================================= T

In [18]:
conversations = [
    (
        response['messages'],
        "Ignore any emails from Alice Jones"
    )
]

In [19]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
        
    },
    {
        "name": "triage-ignore", 
        "prompt": store.get(("lance",), "triage_ignore").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"

    },
    {
        "name": "triage-notify", 
        "prompt": store.get(("lance",), "triage_notify").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of"

    },
    {
        "name": "triage-respond", 
        "prompt": store.get(("lance",), "triage_respond").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to"

    },
]

updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)

for i, updated_prompt in enumerate(updated):
    old_prompt = prompts[i]
    if updated_prompt['prompt'] != old_prompt['prompt']:
        name = old_prompt['name']
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("lance",),
                "agent_instructions",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-ignore":
            store.put(
                ("lance",),
                "triage_ignore",
                {"prompt":updated_prompt['prompt']}
            )
        else:
            print(f"Encountered {name}, implement the remaining stores!")

updated triage-ignore


In [20]:
email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "Lim Yoona <yoona@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """
    	Hi Yoon,

		Urgent issue - your service is down. Is there a reason why.
    """,
}

response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "lance"}}
)
for m in response["messages"]:
    m.pretty_print()

🚫 Classification: IGNORE - This email can be safely ignored


In [21]:
store.get(("lance",), "triage_ignore").value['prompt']

'Ignore marketing newsletters, spam emails, mass company announcements, and any emails from Alice Jones.'